# Proceso de Comunicación

### Angel Alvarez

In [1]:
# Librerias Qiskit
from qiskit import *
from qiskit import QuantumCircuit, transpile, assemble, Aer, IBMQ
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
from qiskit.extensions import *
# Plots
from qiskit.visualization import plot_histogram as plth


from random import sample , choice

In [2]:
def GHZ_init(n):
    """
    Esta funcion genera una copuerta de n 
    qubits para transformar a estados GHZ.
    """
    qc = QuantumCircuit(n)

    #Estado GHZ
    qc.h(0)
    for i in range(1, n):
        qc.cx(0, i)
    
    GHZ = qc.to_gate()
    GHZ.name = '$GHZ$'
    
    return GHZ

def U_b_operation(b):
    """
    Esta función toma un valor binario de 
    longitud 2 o 1 para realizar la codifi-
    cacion por transformacion U_{b_i} o U_{b_j}.
    """
    qc = QuantumCircuit(1)
    
    
    if b == '01' or b == '1':
        qc.x(0)

    elif b == '10':
        qc.y(0)
    
    elif b == '11':
        qc.z(0)
        
    U_b = qc.to_gate()
    U_b.name  = '$U_{' + b + '}$'
    
    return U_b

def X_Measure():
    """
    Esta función genera una compuerta para la medida sobre las bases de X.
    """
    qc = QuantumCircuit(1)
    
    qc.h(0)
    
    X_M = qc.to_gate()
    X_M.name = '$X$ Measure'
    
    return X_M

def GHZ_Measure(n):
    """
    Esta función genera una compuerta para la medida sobre las bases de GHZ_n.
    """
    qc = QuantumCircuit(n)
    
    for i in range(1,n):
        qc.cx(0,n-i)
    qc.h(0)
    
    
    GHZ_M = qc.to_gate()
    GHZ_M.name = 'GHZ Measure'
    
    return GHZ_M

In [3]:
def Decodification(res, n_users, complete_coop):
    """
    Esta función toma el resultado del circuito y decodifica el mensaje de acuerdo a la
    cantidad de usuarios y el modo de trabajo (completa o parcial).
    """
    
    # Decoficacion para dos usuarios
    if n_users == 2:
        if complete_coop: # Cooperación completa
            if res == '000' or res == '101':
                return '00'
            elif res == '001' or res == '100':
                return '11'
            elif res == '010' or res == '111':
                return '01'
            elif res == '011' or res == '110':
                return '10'
            
        else: # Cooperación parcial
            if res == '000' or res == '011':
                return '00'
            elif res == '001' or res == '010':
                return '11'
            elif res == '100' or res == '111':
                return '01'
            elif res == '101' or res == '110':
                return '10'
            
    # Decoficacion para tres usuarios
    if n_users == 3:
        if complete_coop: # Cooperación completa
            if res == '0000' or res == '1001':
                return '000'
            elif res == '0010' or res == '1011':
                return '001'
            elif res == '0100' or res == '1101':
                return '010'
            elif res == '0110' or res == '1111':
                return '011'
            elif res == '0101' or res == '1100':
                return '100'
            elif res == '0111' or res == '1110':
                return '101'
            elif res == '0001' or res == '1000':
                return '110'
            elif res == '0011' or res == '1010':
                return '111'

        else: # Cooperación parcial
            if res == '0000' or res == '1001':
                return '000'
            elif res == '0010' or res == '1011':
                return '001'
            elif res == '0110' or res == '1111':
                return '010'
            elif res == '0100' or res == '1101':
                return '011'
            elif res == '0111' or res == '1110':
                return '100'
            elif res == '0101' or res == '1100':
                return '101'
            elif res == '0001' or res == '1000':
                return '110'
            elif res == '0011' or res == '1010':
                return '111'
        
        

def Print_Measures(res,n_users ,complete_coop):
    """
    Esta función publica los resultados de las 
    medidas de los usuarios y el QS de acuerdo
    a la cantida de usuarios y modo de trabajo.
    """
    
    
    if n_users == 2:
        if complete_coop:
            print('Complete measure:', res)
            print("User measured:",res[0])
            print("QS measured: ",  res[1:3])
        else:
            print('Complete measure:', res)
            print("QS measured:", res[-1])
            print("User measured: ",  res[0:2])
            
    if n_users == 3:
        if complete_coop:
            print('Complete measure:', res)
            print("QS measured:",res[1:4])
            print("User measured: ",  res[0])
        else:
            print('Complete measure:', res)
            print("QS measured:",res[1:4])
            print("User measured: ",  res[0])

In [15]:
def Communication_Between_Users(n_users, bi, bj, complete_coop, info=False):
    
    """
    La función mas importante del código. Esta se encarga de emular 
    la comunicación entre los usuarios y  el QS de acuerdo al modo 
    de trabajo. Sus partes son:
        * Construcción del circuito.
        * Medida del circuito
        * Decodificación del circuito
        * Retorno del mensaje decodificado o retorno del circuito
    """
    
    """Construcción del circuito"""
    
    n_qubits = n_users + 1 
    
    # Circuit init
    qr = QuantumRegister(n_qubits, 'q')
    cr = ClassicalRegister(n_qubits, 'c')
    qc = QuantumCircuit(qr, cr)

    # GHZ init
    GHZ = GHZ_init(n_qubits)
    qc.append(GHZ, range(n_qubits))

    # Operacion de u_i
    U_bi = U_b_operation(bi)
    qc.append(U_bi, [1])
    
    # Operacion de u_j
    if n_users == 3:
        U_bj = U_b_operation(bj)
        qc.append(U_bj, [2])
    
    qc.barrier()
    
    
    """Medida del circuito"""
    
    # Medidas de X y GHZ
    X_M = X_Measure()
    GHZ_M = GHZ_Measure(n_users) # Bell Measure para n = 2 y GHZ para n = 3
    
    if n_users == 2:
        if complete_coop:
            qc.append(GHZ_M, [0,1])
            qc.append(X_M, [2])
        else:
            qc.append(X_M, [0])
            qc.append(GHZ_M, [1,2])
        # Medidas en Z
        qc.measure(range(n_qubits),range(n_qubits))
    
    if n_users == 3:
        if complete_coop:
            qc.append(GHZ_M, [0,1,2])
            qc.append(X_M, [3])
            # Medidas en Z
            qc.measure([3,2,1,0],[3,2,1,0])
        else:
            qc.append(X_M, [0])
            qc.append(GHZ_M, [1,2,3])
            # Medidas en Z
            qc.measure([0,1,2,3],[0,3,2,1])
            
    """Proceso y corrida del circuito"""
    
    sim = Aer.get_backend('qasm_simulator')
    results = execute(qc, backend = sim, shots = 1).result()
    counts = results.get_counts()
    
    if n_users == 2:
        RESULT = list(counts.keys())[0]
    if n_users == 3:
        RESULT = list(counts.keys())[0][::-1]
    
    
    """Decodificación del mensaje"""
    
    message = Decodification(RESULT,n_users, complete_coop)
    
    
    # Parte auxiliar para ver los resultados de los usurios sin devolver nada.
    if not info:
        return message
    else :
        Print_Measures(RESULT, n_users,complete_coop)
        print('Sent Message: ', bi if n_users == 2 else bi+bj )
        print(f'Decoded Message: {message}')
       


In [18]:
Communication_Between_Users(n_users=3, bi='11', bj='0',complete_coop=True, info=True)

Complete measure: 0001
QS measured: 001
User measured:  0
Sent Message:  110
Decoded Message: 110


In [5]:
def Test_Communication(n_users, mode, n):
    """
    Esta función corre el proceso de certificado de la red de comunicación.
    """
    
    true = 0
    n = 1000
    for i in range(n):
        ui_m = sample(['00','01','10','11'],1)[0]
        uj_m = sample(['0','1'],1)[0]

        ul_m = Communication_Between_Users(n_users, ui_m, uj_m , complete_coop=mode,info=False)
        
        if n_users == 2:
            if ui_m == ul_m:
                true = true + 1
        
        if n_users == 3: 
            if (ui_m + uj_m == ul_m ):
                true = true + 1
    
    result = true/n*100
    
    if mode: complete = 'COMPLETE'
    else: complete = 'PARTIAL'
        
    print("\n")
    print(f"Mensajes enviados: {n}" )
    print(f"Usuarios: {n_users}")
    print(f"Efectividad del mensaje en modo {complete} es: {result} \n")


In [6]:
Test_Communication(n_users=2, mode=False, n=1000)
Test_Communication(n_users=2, mode=True, n=1000)
Test_Communication(n_users=3, mode=False, n=1000)
Test_Communication(n_users=3, mode=True, n=1000)



Mensajes enviados: 1000
Usuarios: 2
Efectividad del mensaje en modo PARTIAL es: 100.0 



Mensajes enviados: 1000
Usuarios: 2
Efectividad del mensaje en modo COMPLETE es: 100.0 



Mensajes enviados: 1000
Usuarios: 3
Efectividad del mensaje en modo PARTIAL es: 100.0 



Mensajes enviados: 1000
Usuarios: 3
Efectividad del mensaje en modo COMPLETE es: 100.0 

